# Brochure Generator

In [1]:
# Imports
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

print('The libraries have been imported successfully.')

The libraries have been imported successfully.


In [2]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕████████████████▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 56bb8bd4

In [3]:
# Initialize openai
openai = OpenAI(base_url = 'http://localhost:11434/v1',
                api_key = 'ollama')

In [34]:
# A Webpage class

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    '''
    A utility class to represent a Website that we have scraped, now with links
    '''
    
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers = headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else 'No title found'
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]
        
    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [35]:
ed = Website('https://edwarddonner.com')
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/

In [47]:
# Create link system prompt
link_system_prompt = 'You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n'
link_system_prompt += 'You should respond in JSON as in this example:'
link_system_prompt += '''
{
    'links': [
        {'type': 'about page', 'url': 'https://full.url/goes/here/about'},
        {'type': 'careers page', 'url': 'https://another.full.url/careers'}
    ]
}
'''

In [48]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    'links': [
        {'type': 'about page', 'url': 'https://full.url/goes/here/about'},
        {'type': 'careers page', 'url': 'https://another.full.url/careers'}
    ]
}



In [49]:
# Function user prompt
def get_links_user_prompt(website):
    user_prompt = f'Here is the list of links on the website of {website.url} - '
    user_prompt += 'please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
        Do not include Terms of Service, Privacy, email links.\n'
    user_prompt += 'Links (some might be relative links):\n'
    user_prompt += '\n'.join(website.links)
    return user_prompt

In [50]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format.         Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/

In [51]:
# Function to get the most relevant links
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = 'llama3.2',
        messages = [
            {'role': 'system', 'content': link_system_prompt},
            {'role': 'user', 'content': get_links_user_prompt(website)}
        ],
        response_format = {'type': 'json_object'}
    )
    try:
        result = response.choices[0].message.content
        return json.loads(result)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON response: {e}")
        return {}

In [52]:
huggingface = Website('https://huggingface.co')
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/hexgrad/Kokoro-82M',
 '/microsoft/phi-4',
 '/openbmb/MiniCPM-o-2_6',
 '/NovaSky-AI/Sky-T1-32B-Preview',
 '/deepseek-ai/DeepSeek-V3',
 '/models',
 '/spaces/hexgrad/Kokoro-TTS',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces/FaceOnLive/Face-Search-Online',
 '/spaces/lllyasviel/iclight-v2',
 '/spaces/stabilityai/stable-point-aware-3d',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/NovaSky-AI/Sky-T1_data_17k',
 '/datasets/HumanLLMs/Human-Like-DPO-Dataset',
 '/datasets/DAMO-NLP-SG/multimodal_textbook',
 '/datasets/cfahlgren1/react-code-instructions',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transforme

In [53]:
get_links(huggingface.url)

{'links': [{'type': 'Company page', 'url': 'https://huggingface.co/'},
  {'type': 'About page', 'url': 'https://huggingface.co/about'},
  {'type': 'Blog', 'url': 'https://blog.huggingface.co'},
  {'type': 'GitHub repository', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn company page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Terms of Service alternative: Hugging Face Blog',
   'url': 'https://blog.huggingface.co'}]}

In [61]:
# Function to get all details about all links
def get_all_details(url):
    result = "Landing page:\n"
    try:
        result += Website(url).get_contents()
    except requests.exceptions.RequestException as e:
        result += f"Error fetching the landing page: {e}\n"
        return result

    links = get_links(url)
    print("Found links:", links)

    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        
        try:
            result += Website(link["url"]).get_contents()
        except requests.exceptions.RequestException as e:
            result += f"Error fetching link {link['url']}: {e}\n"
            continue  # Skip this link and continue with the next one

    return result

In [62]:
print(get_all_details(huggingface.url))

Found links: {'links': [{'type': 'Company page', 'url': 'https://huggingface.co/'}, {'type': 'About page', 'url': 'https://brand.huggingface.co'}, {'type': 'Careers/Jobs page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'Blog page', 'url': 'https://blog.huggingface.co'}, {'type': 'News/Status page', 'url': 'https://status.huggingface.co'}, {'type': 'GitHub repository page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter account', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn company profile', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Trending on
this week
Models
hexgrad/Kokoro-82M
Updated
1 day ago
•
20.6k
•
1

In [63]:
# Create system prompt
system_prompt = 'You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information.'

In [68]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f'You are looking at a company called: {company_name}\n'
    user_prompt += f'Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n'
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [69]:
get_brochure_user_prompt('HuggingFace', huggingface.url)

Found links: {'links': [{'type': 'About page', 'url': 'https://huggingface.co/brand'}, {'type': 'Company page', 'url': 'https://huggingface.co/'}, {'type': 'Careers/Jobs page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'Datasets page', 'url': 'https://discuss.huggingface.co/c/datasets/23423'}, {'type': 'Models page', 'url': 'https://huggingface.co/models'}, {'type': 'Spaces page', 'url': 'https://status.huggingface.co/'}, {'type': 'Community page', 'url': 'https://github.com/huggingface'}, {'type': 'Blog page', 'url': 'https://blog.huggingface.co/'}, {'type': 'Jobs and Careers page', 'url': 'https://apply.workable.com/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\nhexgrad/Kokoro-82M\nUpdated\n1 day ago\n•\n20.6k\n•\n1.82k\nmicrosoft/phi-4\nUpdated\n9 days ago\n•\n100k\n•\n1.4k\nopenbmb/MiniCPM-o-2_6\nUpdated\nabout 7 hours ago\n•\n7.05k\n•\n516\nNovaSky-AI/Sky-T1-32B-Preview\nUpdated\n4 days ago\n•\n4.89k\n•\n442\ndeepseek-ai/DeepSeek-V3\nUpdated\n19 days ago\n•\n142k\n•\n1.99k\nBrowse 400k+ models\nSpaces\nRunning\non\nZero\n1.02k\n❤️\nKokoro TTS\nNow in 5 languages!\nRunning\non\nZero\n3.08

In [70]:
# Create final brochure
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model = 'llama3.2',
        messages = [
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': get_brochure_user_prompt(company_name, url)}
        ],
        stream = True
    )
    
    response = ''
    display_handle = display(Markdown(''), display_id = True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace('```', '').replace('markdown', '')
        update_display(Markdown(response), display_id = display_handle.display_id)

In [71]:
stream_brochure('HuggingFace', huggingface.url)

Found links: {'links': [{'type': 'About page', 'url': 'https://huggingface.co/about'}, {'type': 'Company page', 'url': 'https://huggingface.co/'}, {'type': 'Careers/Jobs page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'Blog', 'url': 'https://discuss.huggingface.co/'}, {'type': 'GitHub', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face: Empowering the Future of Artificial Intelligence

## About Us

At Hugging Face, we are a community-driven platform dedicated to advancing the field of artificial intelligence. Our mission is to provide a collaborative environment for researchers and developers to create, share, and apply cutting-edge machine learning models.

**Our Vision**

We aim to democratize access to AI by providing an open-source stack, tools, and services that make it easy for anyone to build, deploy, and use models. Our platform covers a wide range of modalities, including text, images, audio, and 3D data.

## What We Offer

* **Models**: Browse over 400k+ pre-trained models, or create your own using our open-source library.
* **Datasets**: Access and share datasets for various tasks, including computer vision, audio, and NLP.
* **Spaces**: Host and collaborate on unlimited public models, datasets, and applications.
* **Compute**: Deploy models on optimized inference endpoints or update Spaces applications to GPU in a few clicks.

## Our Community

Our community is diverse and active, with over 50,000 organizations using our platform. Some of our notable users include AI2, Meta, Amazon Web Services, Google, Intel, Microsoft, and Grammarly.

## Our Open Source Efforts

We are committed to building the foundation of ML tooling with the community. We offer a range of open-source projects, including:

* **Transformers**: State-of-the-art ML library for PyTorch, TensorFlow, and JAX.
* **Diffusers**: State-of-the-art diffusion models for image and audio generation in PyTorch.
* **Safetensors**: Simple and safe way to store and distribute neural networks weights.

## Join Our Community

Ready to accelerate your ML journey? Sign up for our platform and start building, sharing, and applying cutting-edge machine learning models. Whether you're a researcher, developer, or entrepreneur, we invite you to join our community and shape the future of AI together.